In [1]:
import inspect
from typing import *

In [34]:
class MementorItemAccess:
    def __init__(self, key: str, default: Any = None):
        self.key = key
        self.default = default
    
    def __get__(self, obj: "Mementor", *args, **kwargs):
        return getattr(obj, f"__{self.key}", self.default)

    def __set__(self, obj: "Mementor", value: Any):
        state = getattr(obj, "__STATE")
        state[self.key] = value
        setattr(obj, f"__{self.key}", value)



class MementoMetaclass(type):
    def __new__(cls, name: str, bases: Tuple[type, ...], data: Dict[str, Any], max_size: int = -1):
        if "__annotations__" in data:
            annotations = data.pop("__annotations__")
            for key, dtype in annotations.items():
                assert not key.startswith("_") and key != "STATE", f"Invalid name: {key}"
                default_val = None
                if key in data:
                    default_val = data.pop(key)

                data[key] = MementorItemAccess(key, default_val)
            
        return super().__new__(cls, name, bases, data)
    
    
class Memento(metaclass=MementoMetaclass):
    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls, *args, **kwargs)
        setattr(instance, "__STATE", {})
        return instance

In [42]:
class ConcreteMemento(Memento):
    name: str = "hello"

In [43]:
a = ConcreteMemento()

In [44]:
a.name = "a"

In [45]:
a.__STATE

{'name': 'a'}

In [46]:
a.name = "b"

In [47]:
a.__STATE

{'name': 'b'}

In [48]:
a.greet()

{'name': 'b'}
